In [5]:
def clean_text(text):
    text = re.sub(r'\n+', ' ', text)  # Remove newlines
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = text.strip()  # Remove leading/trailing spaces
    return text

In [7]:
from Bio import Entrez
import re
# Set your email (required by NCBI)
Entrez.email = "knssriharshith11@gmail.com"

# Define a list of biomedical search terms
search_terms = [
    "health", "disease", "treatment", "epidemiology", "clinical trials", "cardiology",
    "oncology", "genetics", "genomics", "biotechnology", "mental health", "COVID-19",
    "public health", "pharmaceutical research"
]

# Collect abstracts for each search term
all_abstracts = []

for term in search_terms:
    # Fetch PubMed IDs (PMIDs) for each search term
    handle = Entrez.esearch(db="pubmed", term=term, retmax=100)
    record = Entrez.read(handle)
    id_list = record["IdList"]
    
    # Fetch the abstracts using PubMed IDs
    fetch_handle = Entrez.efetch(db="pubmed", id=id_list, rettype="abstract", retmode="text")
    abstract_data = fetch_handle.read()
    
    # Clean and append the abstracts to the list
    cleaned_abstracts = [clean_text(abstract) for abstract in abstract_data.split('\n') if abstract]
    all_abstracts.extend(cleaned_abstracts)

# Save or process the abstracts as needed
print(f"Collected {len(all_abstracts)} abstracts")


Collected 63936 abstracts


In [8]:
tokenized_data = tokenizer(all_abstracts, truncation=True, padding="max_length", max_length=512)

NameError: name 'tokenizer' is not defined

In [9]:

import torch
from transformers import BioGptForCausalLM, BioGptTokenizer,TrainingArguments, Trainer

# Load BioGPT model and tokenizer
model_name = "microsoft/biogpt"  # Pre-trained BioGPT model from Hugging Face
tokenizer = BioGptTokenizer.from_pretrained(model_name)
model = BioGptForCausalLM.from_pretrained(model_name)


# Tokenize the biomedical abstracts (already collected and preprocessed)
tokenized_data = tokenizer(all_abstracts, truncation=True, padding="max_length", max_length=512, return_tensors="pt")

# Prepare the data for training
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, tokenized_data):
        self.input_ids = tokenized_data['input_ids']
        self.attention_mask = tokenized_data['attention_mask']
        
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx]
        }

# Initialize the dataset
train_dataset = CustomDataset(tokenized_data)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',            # Output directory for model checkpoints
    overwrite_output_dir=True,
    num_train_epochs=3,                # Number of training epochs
    per_device_train_batch_size=4,     # Batch size per device during training
    save_steps=10_000,                  # Save checkpoint every 10,000 steps
    save_total_limit=2,                # Limit the number of saved checkpoints
    logging_dir='./logs',              # Directory for logs
    logging_steps=500,                 # Log every 500 steps
    learning_rate=5e-5,                # Learning rate for fine-tuning
)

# Initialize the Trainer with the model, training args, and dataset
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer
)

# Start fine-tuning the model
trainer.train()

# Save the fine-tuned model
trainer.save_model("./bio_gpt_finetuned")
tokenizer.save_pretrained("./bio_gpt_finetuned")


vocab.json:   0%|          | 0.00/927k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/696k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/595 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

c:\Users\Harsha\AppData\Local\Programs\Python\Python310\lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


  0%|          | 0/47952 [00:00<?, ?it/s]

ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.